This tutorial is based on sklearn's official tutorial [Working With Text Data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#bags-of-words).

# Loading the 20 newsgroups dataset
The dataset is called “Twenty Newsgroups”. Here is the official description, quoted from the [website](http://people.csail.mit.edu/jrennie/20Newsgroups/):

The 20 Newsgroups data set is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups. To the best of our knowledge, it was originally collected by Ken Lang, probably for his paper “Newsweeder: Learning to filter netnews,” though he does not explicitly mention this collection. The 20 newsgroups collection has become a popular data set for experiments in text applications of machine learning techniques, such as text classification and text clustering.

In the following we will use the built-in dataset loader for 20 newsgroups from scikit-learn. Alternatively, it is possible to download the dataset manually from the website and use the [sklearn.datasets.load_files](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html#sklearn.datasets.load_files) function by pointing it to the 20news-bydate-train sub-folder of the uncompressed archive folder.

In order to get faster execution times for this first example we will work on a partial dataset with only 4 categories out of the 20 available in the dataset:

In [51]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

The returned dataset is a scikit-learn “bunch”: a simple holder object with fields that can be both accessed as python dict keys or object attributes for convenience, for instance the target_names holds the list of the requested category names:

In [8]:
twenty_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

Mappings between target_names and integer label

In [33]:
for i, x in enumerate(twenty_train.target_names):
    print(f'{x:25} {i:5}')

alt.atheism                   0
comp.graphics                 1
sci.med                       2
soc.religion.christian        3


### Let’s print the first loaded file:

In [34]:
print('\n'.join(twenty_train['data'][0].split('\n')))
print(twenty_train.target_names[twenty_train.target[0]], twenty_train.target[0])

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.

comp.graphics 1


# Extracting features from text files¶
## Bag of Words
The most intuitive way to do so is to use a bags of words representation:

* Assign a fixed integer id to each word occurring in any document of the training set (for instance by building a dictionary from words to integer indices).

* For each document #i, count the number of occurrences of each word w and store it in X[i, j] as the value of feature #j where j is the index of word w in the dictionary.

<img src='images/bag_of_words.jpeg'>

## Tokenizing text with scikit-learn

Text preprocessing, tokenizing and filtering of stopwords are all included in [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer), which builds a dictionary of features and transforms documents to feature vectors:

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

[CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) supports counts of N-grams of words or consecutive characters. Once fitted, the vectorizer has built a dictionary of feature indices:

In [37]:
count_vect.vocabulary_.get('algorithm')

4690

In [1]:
# count_vect.vocabulary_

## From occurrences to frequencies

Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.

To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.

Another refinement on top of tf is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

This downscaling is called [tf–idf](https://en.wikipedia.org/wiki/Tf-idf) for “Term Frequency times Inverse Document Frequency”.

Both tf and tf–idf can be computed as follows using [TfidfTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer):

<img src='images/TF-IDF.png'>

In [48]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

## Train a classifier

Let's try a linear [support vector machine (SVM)](https://scikit-learn.org/stable/modules/svm.html#svm)

<img src='images/svm.png'>

In [54]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import numpy as np

text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                           max_iter=5, tol=None)),
 ])

text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(twenty_test['data'])
print(f'Test Accuracy: {np.mean(predicted == twenty_test.target)}')

Test Accuracy: 0.9101198402130493


### Metrics

In [55]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted, target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.80      0.87       319
         comp.graphics       0.87      0.98      0.92       389
               sci.med       0.94      0.89      0.91       396
soc.religion.christian       0.90      0.95      0.93       398

              accuracy                           0.91      1502
             macro avg       0.91      0.91      0.91      1502
          weighted avg       0.91      0.91      0.91      1502



# Parameter tuning using grid search

In [58]:
from sklearn.model_selection import GridSearchCV
parameters = {
     'vect__ngram_range': [(1, 1), (1, 2)],
     'tfidf__use_idf': (True, False),
     'clf__alpha': (1e-2, 1e-3),
 }

gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

### Cross Validation
<img src='images/crossValidation.png'>

### Grid Search
<img src='images/gs.jpg'>

In [59]:
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

In [62]:
print(f'Best Score: {gs_clf.best_score_}')
for param_name in sorted(parameters.keys()):
     print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

Best Score: 0.9641116526362428
clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)


### Summary Reports

In [66]:
import pandas as pd
pd.DataFrame(gs_clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__alpha,param_tfidf__use_idf,param_vect__ngram_range,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.105948,0.024078,0.250149,0.012674,0.01,True,"(1, 1)","{'clf__alpha': 0.01, 'tfidf__use_idf': True, '...",0.904867,0.915929,0.858407,0.887168,0.908686,0.894993,0.020614,6
1,3.668365,0.059414,0.586363,0.016104,0.01,True,"(1, 2)","{'clf__alpha': 0.01, 'tfidf__use_idf': True, '...",0.935841,0.926991,0.891593,0.911504,0.913140,0.915817,0.015099,4
2,1.078371,0.012068,0.232094,0.010825,0.01,False,"(1, 1)","{'clf__alpha': 0.01, 'tfidf__use_idf': False, ...",0.820796,0.787611,0.747788,0.803097,0.781737,0.788214,0.024345,8
3,3.685271,0.114849,0.607813,0.022694,0.01,False,"(1, 2)","{'clf__alpha': 0.01, 'tfidf__use_idf': False, ...",0.825221,0.809735,0.774336,0.823009,0.804009,0.807266,0.018295,7
4,1.139356,0.036813,0.290371,0.010768,0.001,True,"(1, 1)","{'clf__alpha': 0.001, 'tfidf__use_idf': True, ...",0.964602,0.962389,0.957965,0.964602,0.971047,0.964112,0.004222,1
5,4.239864,0.019556,0.575046,0.073604,0.001,True,"(1, 2)","{'clf__alpha': 0.001, 'tfidf__use_idf': True, ...",0.966814,0.962389,0.951327,0.955752,0.959911,0.959238,0.005342,2
6,1.284911,0.078157,0.308939,0.055807,0.001,False,"(1, 1)","{'clf__alpha': 0.001, 'tfidf__use_idf': False,...",0.915929,0.896018,0.911504,0.904867,0.935412,0.912716,0.013153,5
7,3.018627,0.164027,0.323899,0.047419,0.001,False,"(1, 2)","{'clf__alpha': 0.001, 'tfidf__use_idf': False,...",0.931416,0.922566,0.918142,0.926991,0.933185,0.926451,0.005556,3
